In [92]:
import os
import re
import json
import time
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [51]:
def get_query(city_name):
    query = str(city_name.encode('utf-8'))[1:].replace("'", "").replace("\\x", "%25").upper()
    return query

def get_sight_url(response):
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    tags = soup.find_all(attrs={"data-beacon": "SC_resault"})
    sight_url = ""
    get_url = False
    if len(tags) != 0:
        sight_url = tags[0]["href"] + "-jingdian"
        get_url = True
    return (sight_url, get_url)

def get_proxy():
    return requests.get("http://127.0.0.1:5010/get/").json()

def delete_proxy(proxy):
    requests.get("http://127.0.0.1:5010/delete/?proxy={}".format(proxy))

def getResponse(base_url, headers):
    # ....
    retry_count = 5
    proxy = get_proxy().get("proxy")
    while retry_count > 0:
        try:
            html = requests.get(base_url, headers=headers, proxies={"http": "http://{}".format(proxy)})
            # 使用代理访问
            return html
        except Exception:
            retry_count -= 1
    # 删除代理池中代理
    delete_proxy(proxy)
    return None
    
data_folder = r"./original_data"
data_file_names = os.listdir(data_folder)
city_names = map(lambda x: x.split('.')[0], data_file_names)
base_url = r"https://travel.qunar.com/search/all/"
params = {}
headers = {}
with open("./output/sight_url.txt", 'a', encoding="utf-8") as f:
    for city_name in city_names:
        print(city_name)
        ua = UserAgent()
        headers["User-Agent"] = ua.random
        url = base_url + get_query(city_name)
        response = getResponse(url, headers)
        print(response.status_code)
        (sight_url, get_url) = get_sight_url(response)
        f.write(city_name)
        f.write("\n")
        f.write(sight_url)
        f.write("\n")
        print(sight_url)
        time.sleep(1)

七台河
200
https://travel.qunar.com/p-cs300161-qitaihe-jingdian
万宁
200
https://travel.qunar.com/p-cs300187-wanning-jingdian
三亚
200
https://travel.qunar.com/p-cs300188-sanya-jingdian
三明
200
https://travel.qunar.com/p-cs300015-sanming-jingdian
三沙
200
https://travel.qunar.com/p-cs298775-sansha-jingdian
三门峡
200
https://travel.qunar.com/p-cs300177-sanmenxia-jingdian
上海
200
https://travel.qunar.com/p-cs299878-shanghai-jingdian
上饶
200
https://travel.qunar.com/p-cs300025-shangrao-jingdian
东方
200
https://travel.qunar.com/p-cs300146-dongfang-jingdian
东莞
200
https://travel.qunar.com/p-cs299831-dongguan-jingdian
东营
200
https://travel.qunar.com/p-cs300120-dongying-jingdian
中卫
200
https://travel.qunar.com/p-cs300040-zhongwei-jingdian
中山
200
https://travel.qunar.com/p-cs300137-zhongshan-jingdian
临汾
200
https://travel.qunar.com/p-cs299785-linfen-jingdian
临沂
200
https://travel.qunar.com/p-cs300114-linyi-jingdian
临沧
200
https://travel.qunar.com/p-cs299858-lincang-jingdian
临高
200
https://travel.qunar.com/p-

In [63]:
# 提取代理API接口，获取1个代理IP
api_url = "http://v2.api.juliangip.com/dynamic/getips?auto_white=1&num=1&pt=1&result_type=text&split=1&trade_no=1129780214579896&sign=5e8e521b200e03ade682e8906972d00e"

# 获取API接口返回的代理IP
proxy_ip = requests.get(api_url).text

# 用户名密码认证(动态代理/独享代理)
username = "13437953591"
password = "4XrQ6jfh"
proxies = {
"http": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": username, "pwd": password, "proxy": proxy_ip},
"https": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": username, "pwd": password, "proxy": proxy_ip},
}

# 白名单方式（需提前设置白名单）
# proxies = {
#     "http": "http://%(proxy)s/" % {"proxy": proxy_ip},
#     "https": "http://%(proxy)s/" % {"proxy": proxy_ip},
# }

# 要访问的目标网页
target_url = "https://www.juliangip.com/api/general/Test"

# 使用代理IP发送请求
response = requests.get(target_url, proxies=proxies)

# 获取页面内容
if response.status_code == 200:
    print(response.text)

{"ip":"110.82.6.157","state":"ok"}


In [ ]:
"<div class="b_paging"><em class="page cur">1</em><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-2">2</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-3">3</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-4">4</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-5">5</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-6">6</a><em class="dot">...</em><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-199">199</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-200">200</a><a class="page next" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-2">下一页</a></div>"

# 高铁数据爬取

## 获取所有高铁页面链接

In [407]:
url = "https://train.hao86.com/%E9%AB%98%E9%80%9F/"
headers = {}
ua = UserAgent()
headers["User-Agent"] = ua.random
response = requests.get(url, headers=headers)
html_doc = response.text
soup = BeautifulSoup(html_doc, 'lxml')
uls = soup.find_all("ul", attrs={"id": "scrollbar"})
lis = uls[0].find_all('li')
hsr_data = {
    "hsrs": []
}
for li in lis:
    hsr = {}
    hsr_name = li.contents[0]["href"].replace("/", "")
    hsr["name"]= hsr_name
    hsr["url"] = "https://train.hao86.com" + li.contents[0]["href"]
    hsr_data["hsrs"].append(hsr)
with open("./output/hsr_data.json",'w+') as f:
    json.dump(hsr_data, f)

## 爬取每个页面的内容

In [417]:
def get_stations_table(response):
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    divs = soup.find_all("div", attrs={"class": "main_conter_Article clearfix"})
    get_succ = False
    stations_table = None
    pattern = re.compile(r"途经车站")
    if len(divs) != 0:
        for div in divs:
            for i in range(len(div.contents)):
                child = div.contents[i]
                if child !="\n" and len(child.find_all(string=pattern)) > 0:
                    get_succ = True
                    stations_table = div.find_all("div", attrs={"class": "station"})[0].contents
                    stations_table = list(filter(lambda x: x!="\n", stations_table))[0]
                    break
    return (get_succ, stations_table)

def get_stations(stations_table):
    trs = stations_table.find_all("tr")[1:]
    stations = []
    for tr in trs:
        station = tr.find_all("td")[1].a.string.strip()
        stations.append(station)
    return stations

with open("./output/hsr_data.json",'r+') as f:
    hsr_data = json.load(f)
hsrs = hsr_data["hsrs"]
failed_count = 0
headers = {}
for hsr in hsrs:
    url = hsr["url"]
    ua = UserAgent()
    headers["User-Agent"] = ua.random
    response = getResponse(url, headers=headers)
    if response.status_code != 200:
        print("请求失败！")
    else:
        response.encoding = "utf-8"
        (get_succ, stations_table) = get_stations_table(response)
        if get_succ:
            stations = get_stations(stations_table)
            hsr["stations"] = stations
        else:
            failed_count += 1
            print(f"未找到{hsr["name"]}的途经车站。")
    time.sleep(1)

with open("./output/hsr_data.json",'w+') as f:
    json.dump(hsr_data, f)

未找到G185的途经车站。
未找到G1813的途经车站。
未找到G1822的途经车站。
未找到G1826的途经车站。
未找到G1828的途经车站。
未找到G1832的途经车站。
未找到G1852的途经车站。
未找到G1862的途经车站。
未找到G1866的途经车站。
未找到G1869的途经车站。
未找到G1870的途经车站。
未找到G1874的途经车站。
未找到G1880的途经车站。
未找到G1887的途经车站。
未找到G1896的途经车站。
未找到G1865的途经车站。
未找到G701的途经车站。
未找到G702的途经车站。
未找到G703的途经车站。


AttributeError: 'NoneType' object has no attribute 'string'

In [435]:
len(list(filter(lambda x: "stations" in x, hsr_data["hsrs"])))

44

{'name': 'G18',
 'url': 'https://train.hao86.com/G18/',
 'stations': ['上海虹桥', '南京南', '徐州东', '济南西', '北京南']}